In [ ]:
import nemo.collections.asr as nemo_asr
import numpy as np
from IPython.display import Audio, display
import librosa
import os
import wget
import matplotlib.pyplot as plt
import glob

import librosa
import IPython.display as ipd

We use CTC labels for voice activity detection. To detect speech and non-speech segments in the audio, we use blank and space labels in the CTC outputs. Consecutive labels with spaces or blanks longer than a threshold are considered non-speech segments

In [ ]:
ROOT = os.getcwd()
data_dir = os.path.join(ROOT,'data')
os.makedirs(data_dir, exist_ok=True)

# AUDIO_LIST_FILE = "/mnt/data/amicorpus_lapel/lapel_files/amicorpus_test_wav.scp"
AUDIO_LIST_FILE = "/home/fjia/data/modified_callhome/audio_evallist.txt"

AUDIO_LIST = []   
with open(AUDIO_LIST_FILE, "r") as fp:
    lines = fp.readlines()
    for line in lines:
        AUDIO_LIST.append(line.strip())
                
print(len(AUDIO_LIST))
AUDIO_FILENAME = AUDIO_LIST[0]

# audio, sample_rate = librosa.load(AUDIO_FILENAME)
# ipd.Audio(audio, rate=sample_rate)

In [ ]:
#Load model
asr_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name='stt_en_citrinet_1024_gamma_0_25', strict=False)

In [ ]:
vocab = asr_model.decoder.vocabulary
vocab.append("blank") # bad practice actually need fix
print(len(vocab))
print(vocab)

In [ ]:
files = AUDIO_LIST

In [ ]:
# softmax implementation in NumPy
def softmax(logits):
    e = np.exp(logits - np.max(logits))
    return e / e.sum(axis=-1).reshape([logits.shape[0], 1])

In [ ]:
def write_to_rttm(rttm_output_folder, AUDIO_FILENAME, non_speech, probs):

    frame_offset=offset/time_stride
    speech_labels=[]
    uniq_id = os.path.basename(AUDIO_FILENAME).split('.wav')[0]
    with open(os.path.join(rttm_output_folder, uniq_id+'.rttm'),'w') as f:
        for idx in range(len(non_speech)-1):
            start = (non_speech[idx][1]+frame_offset)*time_stride
            end = (non_speech[idx+1][0]+frame_offset)*time_stride
            
            f.write("SPEAKER {} 1 {:.3f} {:.3f} <NA> <NA> speech <NA>\n".format(uniq_id,start,end-start))
            speech_labels.append("{:.3f} {:.3f} speech".format(start,end))
        if non_speech[-1][1] < len(probs):
            start = (non_speech[-1][1]+frame_offset)*time_stride
            end = (len(probs)+frame_offset)*time_stride
            f.write("SPEAKER {} 1 {:.3f} {:.3f} <NA> <NA> speech <NA>\n".format(uniq_id,start,end-start))
            speech_labels.append("{:.3f} {:.3f} speech".format(start,end))
            
    return speech_labels

In [ ]:
def extract(probs, vocab, threshold):
    # No space for citrinet just blanks
    
    blanks = []
    state = ''
    idx_state = 0
    # TODO change all 1024 to dim
    
    # blank
    if np.argmax(probs[0]) == 1024: #28 hard-coded bad!!
        state = 'blank'

    for idx in range(1, probs.shape[0]):
        current_char_idx = int(np.argmax(probs[idx]))
        if state == 'blank' and current_char_idx != 1024: #28
            # previous is blank but current is not blank
            blanks.append([idx_state, idx-1])
            state = ''
            
        if state == '': # previous is not blank
            if current_char_idx == 1024: #28
                state = 'blank'
                idx_state = idx

    if state == 'blank':
        blanks.append([idx_state, len(probs)-1])
    
    non_speech=list(filter(lambda x:x[1]-x[0]>threshold, blanks)) 
    print(len(blanks), len(non_speech))
    return  non_speech

In [ ]:
def process_one_session(logits, threshold):
    probs = softmax(logits)
    non_speech = extract(probs, vocab, threshold)
    return non_speech, probs

In [ ]:
all_logits = asr_model.transcribe(files, logprobs=True)

In [ ]:
offset = 0.00 #-0.18  Confirmed with Patrice
time_stride = 0.08 #0.02 Checked and Confirmed with Som

threshold = 12 #minimun width to consider non-speech activity 

rttm_output_folder = "/home/fjia/code/NeMo-fei/tutorials/speaker_recognition/" + str(threshold) + "_eval_citri_ch"

In [ ]:
if not os.path.exists(rttm_output_folder):
    os.mkdir(rttm_output_folder)
    
for i in range(len(all_logits)):
    logits = all_logits[i]
    non_speech, probs = process_one_session(logits, threshold)
    AUDIO_FILENAME = files[i]
    speech_labels =  write_to_rttm(rttm_output_folder, AUDIO_FILENAME, non_speech, probs)

In [ ]:
paths2rttm_files = glob.glob(rttm_output_folder + "/*.rttm")
paths2rttm_files

In [ ]:
from nemo.collections.asr.parts.utils.speaker_utils import write_rttm2manifest
output_dir = os.path.join(ROOT, 'citrinet_vad_ch')
os.makedirs(output_dir,exist_ok=True)
oracle_manifest = os.path.join(output_dir, str(threshold)+'eval_manifest.json')
write_rttm2manifest(paths2audio_files=files,
                    paths2rttm_files=paths2rttm_files,
                    manifest_file=oracle_manifest)

In [ ]:
!cat {output_dir}/2eval_manifest.json